# *Importing the Required Libraries*

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

# *Reading the Data from CSV file*

In [2]:
movies_data = pd.read_csv('movies_data.csv')

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.tail()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,0,Documentary,NaN,25975,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,...,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [4]:
movies_data.shape

(4803, 24)

In [5]:
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

# *Selecting the desired features for the movie recommandation system*

In [6]:
selected_features = ['genres','keywords','original_title','overview','tagline','cast','director']

# *Filling all the missing values with the null string*

In [7]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

# *Combining all the features*

In [14]:
combined_feature = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['original_title']+' '+movies_data['overview']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

print(type(combined_feature))

<class 'pandas.core.series.Series'>


In [9]:
combined_feature

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  Newlyweds A newlywed couple's ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      Shanghai Calling When ambitious New York att...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

# *Converting the text data into feature vectors*

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
feature_vector = vectorizer.fit_transform(combined_feature)

In [12]:
print(feature_vector)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 314134 stored elements and shape (4803, 31635)>
  Coords	Values
  (0, 575)	0.05873643177978674
  (0, 720)	0.06729868747117117
  (0, 10038)	0.08379568179288469
  (0, 24689)	0.07816785214619333
  (0, 10317)	0.07825787263520055
  (0, 6808)	0.14751739050825702
  (0, 5453)	0.15246705092715887
  (0, 11125)	0.10888892434049498
  (0, 26179)	0.23793627700487918
  (0, 30377)	0.08465719846824456
  (0, 5773)	0.17049482007089195
  (0, 25988)	0.13117950970501566
  (0, 2136)	0.19956703696582706
  (0, 13885)	0.03567801278828226
  (0, 27939)	0.0802116123909307
  (0, 241)	0.19406307037454443
  (0, 4920)	0.12277035544662564
  (0, 20716)	0.18956600222467193
  (0, 17530)	0.1404215434342093
  (0, 14453)	0.040954350720127826
  (0, 8062)	0.17696618855934157
  (0, 28241)	0.03040298524162499
  (0, 18795)	0.14277242492893585
  (0, 20647)	0.3649403103012484
  (0, 20131)	0.04792463927908853
  :	:
  (4802, 22037)	0.08588485159167093
  (4802, 9441)	0.0749

# *Using Cosine Similarity*

In [33]:
# finding the cosine similarity

similarity = cosine_similarity(feature_vector)

In [34]:
similarity

array([[1.        , 0.0540225 , 0.03084845, ..., 0.02436576, 0.02810208,
        0.00666179],
       [0.0540225 , 1.        , 0.04099378, ..., 0.05361211, 0.03669106,
        0.01553442],
       [0.03084845, 0.04099378, 1.        , ..., 0.0227109 , 0.0425595 ,
        0.01229387],
       ...,
       [0.02436576, 0.05361211, 0.0227109 , ..., 1.        , 0.02988327,
        0.03921517],
       [0.02810208, 0.03669106, 0.0425595 , ..., 0.02988327, 1.        ,
        0.02879873],
       [0.00666179, 0.01553442, 0.01229387, ..., 0.03921517, 0.02879873,
        1.        ]])

In [35]:
similarity.shape

(4803, 4803)

In [36]:
similarity[0]

array([1.        , 0.0540225 , 0.03084845, ..., 0.02436576, 0.02810208,
       0.00666179])

# *Taking name of the movie from the user*

In [15]:
movie_name_user = input("Enter the name of your favourite movie : ")

# *Finding the clossest match of the user's entered movie*

In [16]:
movie_list = movies_data['original_title'].tolist()

In [17]:
movie_list

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

# *Finding the close match*

In [18]:
find_close_match = difflib.get_close_matches(movie_name_user,movie_list)

In [19]:
find_close_match

['King Kong', 'Kingpin', 'Killing Zoe']

In [27]:
if len(find_close_match) == 0:
    print("Sorry not possible to recommand any of the movie\n")
    print("You can try again with another movie\n")

In [28]:
close_match = find_close_match[0]

In [29]:
close_match

'King Kong'

# *Getting the index value of the closed match movie*

In [30]:
index_of_movie = movies_data[movies_data['original_title']==close_match]['index'].values[0]

In [31]:
index_of_movie

np.int64(24)

# *Comparing the similarity score of all the movies of the dataset with the user entered movie*

In [37]:
similarity_score = list(enumerate(similarity[index_of_movie]))

In [38]:
similarity_score

[(0, np.float64(0.02615970786690307)),
 (1, np.float64(0.05575630909438482)),
 (2, np.float64(0.0169172950276458)),
 (3, np.float64(0.03351138820127546)),
 (4, np.float64(0.06314102822310377)),
 (5, np.float64(0.040175558683938115)),
 (6, np.float64(0.040809362307390906)),
 (7, np.float64(0.03752424136148442)),
 (8, np.float64(0.016080633662343486)),
 (9, np.float64(0.022817247687887777)),
 (10, np.float64(0.026431549534179803)),
 (11, np.float64(0.02368471843992424)),
 (12, np.float64(0.052770918874692284)),
 (13, np.float64(0.02503083003802886)),
 (14, np.float64(0.04204737359941054)),
 (15, np.float64(0.05786606376452877)),
 (16, np.float64(0.03868951152474527)),
 (17, np.float64(0.060137988847745634)),
 (18, np.float64(0.061970348005531764)),
 (19, np.float64(0.05825996438414264)),
 (20, np.float64(0.07655251350596232)),
 (21, np.float64(0.03292598148543608)),
 (22, np.float64(0.037140644606968275)),
 (23, np.float64(0.028748015495807765)),
 (24, np.float64(1.0000000000000002)),
 (

In [39]:
len(similarity_score)

4803

In [40]:
sorted_similar_movies = sorted(similarity_score,reverse=True,key=lambda x: x[1])

In [41]:
sorted_similar_movies

[(24, np.float64(1.0000000000000002)),
 (3400, np.float64(0.14342255662941014)),
 (1537, np.float64(0.12269431631793937)),
 (1232, np.float64(0.12004119738713509)),
 (1466, np.float64(0.1190452272966286)),
 (1032, np.float64(0.11614263710498687)),
 (1405, np.float64(0.11476203007423225)),
 (2572, np.float64(0.11377878049157729)),
 (778, np.float64(0.1112481092000824)),
 (353, np.float64(0.11077110452594643)),
 (3375, np.float64(0.11042154328724517)),
 (1774, np.float64(0.11034054844162419)),
 (3311, np.float64(0.1068958573521682)),
 (409, np.float64(0.10646959191290886)),
 (2470, np.float64(0.10472846044591849)),
 (4645, np.float64(0.10388105452100727)),
 (2535, np.float64(0.10297790587917292)),
 (2129, np.float64(0.10291651232911316)),
 (48, np.float64(0.1014704378529044)),
 (53, np.float64(0.09944575041736933)),
 (4754, np.float64(0.09864376713543617)),
 (3305, np.float64(0.09851825169644074)),
 (4110, np.float64(0.09837209216078201)),
 (1996, np.float64(0.09728504165340413)),
 (836,

# *Suggesting the movies for the user*

In [42]:
print("The suggested movie for you are listed below \n")

i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  movie_title = movies_data[movies_data.index==index]['original_title'].values[0]
  if i<=30:
    print(f"{i} -- {movie_title}")
    i+=1

The suggested movie for you are listed below 

1 -- King Kong
2 -- Z 風暴
3 -- Get Shorty
4 -- Morning Glory
5 -- Unfinished Business
6 -- America's Sweethearts
7 -- The Pianist
8 -- Boogie Nights
9 -- Meet Dave
10 -- Tropic Thunder
11 -- Redbelt
12 -- What Just Happened
13 -- Thank You for Smoking
14 -- All That Jazz
15 -- Double Impact
16 -- Sugar Town
17 -- Big Fat Liar
18 -- The Black Hole
19 -- Jack the Giant Slayer
20 -- Indiana Jones and the Kingdom of the Crystal Skull
21 -- The Naked Ape
22 -- The Blood of Heroes
23 -- Inside Deep Throat
24 -- Someone Like You...
25 -- The Forbidden Kingdom
26 -- My Week with Marilyn
27 -- The Last Time I Committed Suicide
28 -- Bowfinger
29 -- Incident at Loch Ness
30 -- Master and Commander: The Far Side of the World


# **Movie Recommandation System**

In [43]:
def recommand_movie(movies_data,movie_list,similarity,movie_name_user):
  find_close_match = difflib.get_close_matches(movie_name_user,movie_list)
  if len(find_close_match)>0:
    close_match = find_close_match[0]
    index_of_movie = movies_data[movies_data['original_title']==close_match]['index'].values[0]
    similarity_score = list(enumerate(similarity[index_of_movie]))
    sorted_similar_movies = sorted(similarity_score,reverse=True,key=lambda x: x[1])
    print("The suggested movie for you are listed below \n")
    i = 1
    for movie in sorted_similar_movies:
      index = movie[0]
      movie_title = movies_data[movies_data.index==index]['original_title'].values[0]
      if i<=30:
        print(f"{i} -- {movie_title}")
        i+=1
  else:
    print("Sorry not possible to recommand any of the movie\n")
    print("You can try again with another movie\n")

In [44]:
movie_name_user = input("Enter the name of your favourite movie : ")
recommand_movie(movies_data,movie_list,similarity,movie_name_user)

The suggested movie for you are listed below 

1 -- Party Monster
2 -- This Is the End
3 -- The Players Club
4 -- Home Alone 2: Lost in New York
5 -- The Cotton Club
6 -- America Is Still the Place
7 -- How to Be Single
8 -- Neighbors 2: Sorority Rising
9 -- Shattered Glass
10 -- Inside Deep Throat
11 -- Basquiat
12 -- Synecdoche, New York
13 -- A Lot Like Love
14 -- The Game
15 -- Dave Chappelle's Block Party
16 -- Vampires
17 -- My Week with Marilyn
18 -- Without a Paddle
19 -- Changing Lanes
20 -- Autumn in New York
21 -- The Hotel New Hampshire
22 -- Final Destination
23 -- 13 Going on 30
24 -- #Horror
25 -- Bizarre
26 -- Confessions of a Teenage Drama Queen
27 -- Madagascar 3: Europe's Most Wanted
28 -- Choke
29 -- Just Looking
30 -- Austin Powers: International Man of Mystery
